# Construire une carte interactive à partir de Glottolog

Dans cet exercice, vous allez produire une carte interactive en HTML, en partant de la page d'une langue (ou famille de langues) de votre choix sur le site [Glottolog](https://glottolog.org/). Pour ce faire, vous allez :
1. Naviguer automatiquement sur le site pour afficher la page de la carte, puis la fenêtre contenant les données correspondantes.
2. Extraire les coordonnées des points au format GeoJSON.
3. Déclarer une carte avec les coordonnées de départ et le fond de carte de votre choix.
4. Créer une couche de données et y ajouter les points dont vous avez obtenu les coordonnées.

## Récupérer les données GeoJSON

### Import des librairies et initialisation d'un `webdriver`

Vous n'avez pas besoin de modifier la cellule ci-dessous, elle contient les modules et librairies Python dont vous aurez besoin pour cette partie du travail. Pour que le programme fonctionne, n'oubliez pas de l'exécuter quand même ! (`ctrl`+`enter` ou `shift`+`enter`)
* `webdriver` de `selenium` simule un navigateur web et permettra de mettre en place une navigation scriptée.
* `time` propose des fonctions relatives au temps, et vous servira ici à mettre le script en pause pour laisser charger les pages web.

In [ ]:
from selenium import webdriver
import time

La cellule ci-dessous peut également rester telle quelle, et initie un objet `webdriver` émulant le navigateur Firefox. Celui-ci correspond à la variable `driver`, que vous pouvez désormais utiiser pour mettre en place une navigation automatique. 

In [ ]:
options = webdriver.FirefoxOptions()
options.headless = True
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
webdrivers_path = "./WebDrivers/"
driver = webdriver.Firefox(webdrivers_path, options=options)

Pour des explications plus détaillées sur l'utilisation de `selenium` et `webdriver`, vous pouvez consulter [cet article](https://tekipaki.hypotheses.org/1475).

**Pour la suite, à vous de jouer !**

###  Accéder à la page web, cliquer sur le bouton pour afficher la carte

Pour commencer, ouvrez votre navigateur préféré et naviguez manuellement sur le site [Glottolog](https://glottolog.org) pour choisir la langue ou famille que vous voulez représenter sur votre carte. Vous trouverez sur [cette page](https://glottolog.org/glottolog/language) une liste des langues disponibles.

Affichez la page correspondant à la langue choisie. Son URL devrait correspondre à `https://glottolog.org/resource/languoid/id/` suivi de l'identifiant de la langue dans la base de Glottolog (4 lettres + 4 chiffres).

Dans la cellule ci-dessous, associez cette URL à la variable `url`.

In [ ]:
# url = ""

La page comporte un bouton *show big map*, tel que montré sur l'image ci-dessous (entouré d'une jolie ellipse orange).

![show big map button](minn1241_webpage_showmapbutton.png "Show big map button (minn2141)")

Associez le lien vers lequel pointe ce bouton à la variable `map_page`. Accédez ensuite à ce lien via votre objet `driver`déclaré plus haut. La fonction de `driver` permettant d'accéder à une page web s'appelle `.get()` et prend l'URL en paramètre. 

In [ ]:
# associer le lien à la variable map_page
# accéder à l'URL avec driver

Si tout se passe bien, votre `driver` accède maintenant à la page sur laquelle est affichée la carte. Mais ça ne suffit pas pour pouvoir générer votre propre carte ensuite. Ce dont vous avez besoin, ce sont les coordonnées géographiques de l'ensemble des points représentés sur la carte.

### Ouvrir la fenêtre contenant les coordonnées des points

Ces données sont disponibles au format GeoJSON, vous pouvez y accéder en cliquant sur le bouton *GeoJSON* en haut à droite de la fenêtre de la carte. Si vous cliquez dessus à la main, puis sur le nom de la langue dans le menu déroulant, vous déclencherez l'ouverture d'une fenêtre modale.

Dans le navigateur que vous avez ouvert pour naviguer manuellement, inspectez le code source pour récupérer l'identifiant de ce bouton *GeoJSON*. Dans votre script, c'est `driver` qui va repérer cet élément pour que vous puissiez ensuite lui donner l'instruction de cliquer dessus.

La fonction de `driver` qui vous permet de repérer un élément donné dans le code source est `.find_element_by_id("id-de-l-element")`. Pour pouvoir l'utiliser ensuite, je vous propose de l'associer à la variable `opener`.

In [ ]:
# associer le bouton GeoJSON à la variable opener

Votre `driver` a maintenant connaissance du bouton *GeoJSON* mais n'a pas encore interagi avec. 

Pour cliquer sur un élément, rien de plus simple, il suffit d'appeler dessus la fonction `.click()` (d'où l'intérêt d'avoir associé le bouton à une variable).

Ce clic va ouvrir le menu déroulant. Par précaution, il vaut mieux laisser un petit temps de repos pour s'assurer que le chargement est bien terminé. La fonction `time.sleep()` prend en paramètre une durée en secondes, et permet de mettre le script en pause. 

In [ ]:
# cliquer sur le bouton GeoJSON
# mettre le script en pause pendant une seconde

Tout ceci est bien entendu invisible, mais tout comme un utilisateur humain qui aurait cliqué sur le bouton *GeoJSON*, `driver` est maintenant capable d'interagir avec les éléments du menu déroulant.

Pour ouvrir la fenêtre modale, il faut cliquer sur le nom de la langue dans le menu déroulant, en suivant la même procédure que pour le bouton *GeoJSON*. L'`id` de l'item du menu sur lequel cliquer sera associé à la variable `geo_container`.

In [ ]:
# associer l'élément du menu déroulant (nom de la langue) à la variable geo_container
# cliquer sur cet élément
# mettre le script en pause pendant une seconde

### Extraire le contenu textuel de la fenêtre modale

La fenêtre modale est maintenant "affichée" pour `driver`. Comme pour les autres éléments, pour interagir avec cette fenêtre, il faut commencer par signaler son existence à votre `driver`. 

Procédez de la même façon qu'avec les boutons pour repérer la fenêtre modale, et associez-la à la variable `geo_contents`. 

In [ ]:
# associer la fenêtre modale à la variable geo_contents

Les données dont vous avez besoin sont à portée de `driver` ! 

Comment les obtenir ? Si vous observez le contenu de `geojson_contents`, par exemple avec `print()` ou avec `type()`, vous pouvez remarquer qu'il s'agit d'un objet `webelement` (`webelement.FirefoxWebElement`, puisque `driver` a ici été initié à partir d'un pilote Firefox). 

In [ ]:
type(geojson_contents)

La [documentation *Web Elements*](https://www.selenium.dev/documentation/webdriver/elements/) indique ce que vous pouvez faire avec cette classe d'objets. 

Parcourez cette documentation pour retrouver comment obtenir le contenu textuel d'un `webelement`. Associez le résultat obtenu à la variable `geojson_data`.

In [ ]:
# associer le contenu textuel de la fenêtre modale à la variable geojson_data

Pour vérifier si les données ont bien été récupérées, utilisez `print(geojson_data)`. Les données au format GeoJSON devraient alors s'afficher.

In [ ]:
print(geojson_data)

Si c'est bien le cas, vous pouvez refermer votre `driver` et passer à la suite.

In [ ]:
driver.quit()

## Construire la représentation cartographique

### Import de la librairie `folium`

Pour produire une carte au format HTML, vous aurez besoin de la librairie `folium`. Celle-ci est importée dans la cellule ci-dessous, qui n'a pas besoin d'être modifiée.

In [ ]:
import folium

### Mettre en place le fond de carte

Avant de représenter les données, il faut d'abord déclarer et paramétrer la carte, via une instance d'objet `Map` de la librairie `folium`. Vous pouvez éventuellement initialiser l'objet sans lui attribuer de paramètre, en l'associant à la variable `ma_carte`.

In [ ]:
# déclarez la variable ma_carte en lui associant une instance d'objet folium Map() sans préciser de paramètre 

Pour obtenir un aperçu de la carte, il suffit d'appeler la variable dans laquelle elle est stockée.

In [ ]:
# prévisualiser la carte

In [ ]:
ma_carte

Vous obtenez bien une carte, mais cette vue par défaut n'est ni satisfaisante ni optimale pour visualiser des données.

Déclarez à nouveau votre carte, en renseignant au moins les paramètres `location` (coordonnées par défaut), `zoom_start` (zoom par défaut, basé sur la valeur de `location`), et `tiles` (fond de carte par défaut).

Pour ce faire, vous pouvez vous aider de [cette page de la documentation de `folium`](https://python-visualization.github.io/folium/modules.html). Si vous ne lisez pas l'anglais, [cet article de *Tekipaki*](https://tekipaki.hypotheses.org/1225) peut également vous apporter des informations.

In [ ]:
# déclarer à nouveau la carte avec ses paramètres

In [ ]:
# prévisualiser à nouveau la carte

C'est (normalement) beaucoup mieux comme ça !

### Déclarer une couche de données

Il est temps d'ajouter à la carte les données récupérées sur Glottolog. Voici comment procéder :
1. Déclarer une couche de données
2. Ajouter les données à la couche
3. Ajouter la couche à la carte

La couche de données correspond à un objet de `folium` nommé `FeatureGroup`. Vous trouverez ses différents paramètres dans [la documentation](https://python-visualization.github.io/folium/modules.html#folium.map.FeatureGroup), mais vous pouvez ici l'initialiser avec seulement le paramètre `name`. 

Pour pouvoir être ajoutée à la carte ensuite il est indispensable de stocker les informations de la couche de données dans une variable ; je vous conseille de la nommer de façon à identifier quelles sont les données correspondantes (vous pouvez par exemple reprendre l'identifiant Glottolog de la langue que vous avez choisie).

In [ ]:
# déclarer une couche de données en renseignant le paramètre name, l'associer à une variable nommée explicitement

### Reporter les coordonnées géographiques dans la couche de données

La couche de données existe maintenant dans le sens où une instance d'objet `FeatureGroup` a été créé, mais elle ne comporte aucune information (à part son nom), et n'est pas reliée à la carte.

Pour rappel, les coordonnées sont actuellement stockées dans la variable `geojson_data`, au format GeoJSON.

Celles-ci correspondent à une série de points. Dans le code, chaque point correspondra à un objet `folium.CircleMarker()` qui prendra en paramètre les coordonnées géographiques `[latitude, longitude]`.

Il n'est pas nécessaire de comprendre l'intégralité des informations, mais il faut au moins comprendre comment identifier :
* Quelle portion de la structure JSON correspond à un point ?
* Dans cette portion, où sont indiquées les coordonnées géographiques ?

Vous pouvez les repérer en parcourant manuellement le contenu de `geojson_data`.

In [ ]:
# afficher le contenu de la variable geojson_data

Il est plus efficace de parser le contenu pour le parcourir automatiquement, ce qui n'est pas possible en l'état car `geojson_data` est en fait une simple chaîne de caractères.

In [ ]:
type(geojson_data)

Python dispose de différents moyens de lire du JSON, voici une façon de faire via `folium`.

Exécutez la cellule ci-dessous, et vérifiez à quel type de données correspond la variable `geo_data`.

In [ ]:
geo_data = folium.GeoJson(geojson_data).data

In [ ]:
# vérifier le type de geo_data

Le type `dict` correspond à un dictionnaire, c'est-à-dire une liste associative.

`geo_data` peut maintenant être parcouru automatiquement avec les méthodes relatives à la classe `dict` ([documentation](https://docs.python.org/3/tutorial/datastructures.html#dictionaries)) ou avec des boucles. Par exemple :

In [ ]:
for key in geo_data:
    print(key)

Vous avez maintenant tout ce qu'il vous faut pour ajouter vos données à la couche déclarée précédemment. 

Sachant qu'un point est ajouté avec la fonction `.add_to(nom_de_la_couche_de_donnees)`, décommentez et complétez le code de la cellule suivante :

In [ ]:
"""
for feature in ...:
    folium.CircleMarker(
        location=...,
    ).add_to(...)
"""

### Ajouter la couche de données à la carte

Le plus dur est fait. La fonction `.add_to()` permet également d'ajouter une couche de données à une carte, en utilisant la syntaxe `nom_de_la_couche_de_donnees.add_to(nom_de_la_carte)`.

In [ ]:
# ajouter la couche de données à la carte

Félicitations, vous avez terminé ! 🎉 

Les point apparaîtront maintenant lorsque vous visualiserez votre carte. Vous pouvez admirer le résultat de votre travail en appelant la variable correspondant à la carte.

In [ ]:
# visualiser la carte

## Aller plus loin

Si vous voulez continuer à vous amuser avec les représentations cartographiques produite par `folium`, voici quelques suggestions sur ce que vous pouvez essayer :

* Exporter la carte dans un fichier HTML
* Tracer d'autres séries de points
* Tracer un ou plusieurs polygones
* Capturer l'image de la carte dans un fichier PNG
* Mettre en place un *layer control*
* Définir des fonctions pour répéter aisément les traitements

Les différents [articles consacrés à la cartographie de données linguistiques sur *Tekipaki*] pourront vous être utiles si besoin.

---

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Licence Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" href="http://purl.org/dc/dcmitype/InteractiveResource" property="dct:title" rel="dct:type">LTTAC_2021_TP_cartographie_linguistique</span> de <a xmlns:cc="http://creativecommons.org/ns#" href="https://tekipaki.hypotheses.org/" property="cc:attributionName" rel="cc:attributionURL">Alexander Delaporte</a> est mis à disposition selon les termes de la <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">licence Creative Commons Attribution -  Partage dans les Mêmes Conditions 4.0 International</a>.<br />Code source disponible : <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/alxdrdelaporte/LTTAC_2021_TP" rel="dct:source">https://github.com/alxdrdelaporte/LTTAC_2021_TP</a>.